# Process SIMBAD HIP results

Many of these HIP stars are in a binary system, unresolved by Hipparcos but resolved by Gaia which will cause cross-matching issue. Usually by the fact that they are missing `FLUX_G` but not `FLUX_V`, `FLUX_J` because Gaia did resolve them. The underlying issue is these binary stars are closed enough to be observed by 2MASS/Hipparcos as a single source hence having V, JHK band magnitude but resolved by Gaia hence not having a G-band magnitude because such single object does not exist in Gaia.

## Data Loading and cross-match HIP and Gaia

Every HIP stars should have a Gaia counterpart unless they are very bright or not a stars.

*Note*: These HIP ids are not strictly stars so we will delete them
- HIP 1902 is a globular cluster NGC 104
- HIP 59018 is an artifact
- HIP 60450 is an artifact
- HIP 95723 is an artifact
- HIP 103992 is a planetary nebular NGC 7009
- HIP 108802 is a composite object / blend
- HIP 54948 is a cluster of star (cant be sure which one they are referring, doubt Hipparcos has resolved them)
- HIP 24647 is HIP 24648
- HIP 29116 is HIP 29119
- HIP 35194 is HIP 35195
- HIP 91906 is HIP 91924
- HIP 98623 is HIP 98625
- HIP 88759 is HIP 88762
- And a few more...

And this is a list of NASA Scientific Visualization Studio missing naked eyes visible stars (HR 2366 is not actually missing from HIP)

HR 2950 is a binary stars resolved in Gaia, (v) means manually checked Gaia has the stars.

missing_stars = HR + [4210 (v), 4375 (v), 4374 (v), 5978 (v), 5977 (v), 4729 (v), 2322 (v), 5343 (v), 2950 (v+v), 1982 (v), 5034 (v), 4619 (v), 1704 (v), 6660 (v), 6263 (v), 2341 (v), 6848 (v)]

To get Gaia data, the ADQL query used is as follow:

```sql
with x as
(
	SELECT G.source_id, H2.original_ext_source_id as hip, G.ra, G.dec, G.parallax, G.parallax_over_error, G.pmra, G.pmdec, G.phot_g_mean_mag, G.bp_rp, G.radial_velocity, G.radial_velocity_error, G.astrometric_params_solved, G.ruwe, G.rv_expected_sig_to_noise, G.phot_g_mean_mag - (0.01426 * POWER(G.bp_rp, 3) - 0.2156 * POWER(G.bp_rp, 2) + 0.01424 * POWER(G.bp_rp, 1) - 0.02704) as v_mag 
    FROM gaiadr3.gaia_source AS G
	LEFT JOIN gaiadr3.hipparcos2_best_neighbour AS H2 ON H2.source_id = G.source_id
	WHERE G.phot_g_mean_mag IS NOT NULL OR H2.source_id IS NOT NULL
)
SELECT * 
FROM x
WHERE (bp_rp IS NOT NULL AND v_mag <= 15.5) OR (bp_rp IS NULL AND phot_g_mean_mag <= 15.5)
```

correspond to file named `1733546104637O-result.fits` used in here (You will get a different filename if you do the same query yourself)


In [ ]:
import hashlib
import pathlib
import re
import struct
import time
import warnings

import astropy.units as u
import pandas as pd
import tqdm
from astropy.coordinates.name_resolve import NameResolveError
from astropy.table import Table
from astroquery.gaia import Gaia

from py.utils import custom_simbad

base_path = pathlib.Path("simbad_query_results")
hip_combined_path = base_path / "hip_combined.dat"
sao_combined_path = base_path / "sao_combined.dat"
hd_combined_path = base_path / "hd_combined.dat"
hr_combined_path = base_path / "hr_combined.dat"
# check if the combined file already exists, if not then raise
for combined_path in [
    hip_combined_path,
    sao_combined_path,
    hd_combined_path,
    hr_combined_path,
]:
    if not combined_path.exists():
        raise FileNotFoundError(
            f"{combined_path} does not exist. Please run simbad_query_hipsaohdhr.py first."
        )
hip_combined_table = Table.read(hip_combined_path, format="ascii")
sao_combined_table = Table.read(sao_combined_path, format="ascii")
hd_combined_table = Table.read(hd_combined_path, format="ascii")
hr_combined_table = Table.read(hr_combined_path, format="ascii")

gaia_id_path = base_path / "gaia_source_id_lookup.csv"
try:
    gaia_id_df = pd.read_csv(gaia_id_path, dtype="Int64")
except FileNotFoundError:
    gaia_id_df = pd.DataFrame(columns=["dr2_sourceid", "dr3_sourceid"], dtype="Int64")
    gaia_id_df.to_csv(gaia_id_path, index=False)
simbad_heriarchy_cache = base_path / "heriarchy_cache"
simbad_heriarchy_cache.mkdir(parents=True, exist_ok=True)
# remove the known bad HIP stars
hip_to_remove = [
    1902,
    24647,
    29116,
    35194,
    54948,
    88759,
    91906,
    98623,
]  # indices of rows
hip_combined_table.remove_rows([i - 1 for i in hip_to_remove])

# regex
re_gaiadr2 = re.compile(r"Gaia DR2\s+(\d+)")
re_gaiadr3 = re.compile(r"Gaia DR3\s+(\d+)")
re_hip = re.compile(r"HIP\s+(\d+)")
re_sao = re.compile(r"SAO\s+(\d+)")
re_hd = re.compile(r"HD\s+(\d+)")
re_hr = re.compile(r"HR\s+(\d+)")

# create an empty dataframe for cross ID of integers
cross_id_df = pd.DataFrame(
    columns=["hip", "gaia_dr3", "component", "sao", "hd", "hr"], dtype="Int64"
)
cross_id_df["component"] = cross_id_df["component"].astype(pd.StringDtype())


def extract_gaia_number(text, strict=False, cone_search=False):
    """
    This function will extract the Gaia DR number from the text. Try Gaia DR3 first, if not found try Gaia DR2.

    Parameters
    ----------
    text : str
        The text to extract the Gaia DR number from
    strict : bool
        If True, only extract Gaia DR3 number. Otherwise, try to extract other Gaia DR number if Gaia DR3 is not found
    cone_search : bool
        If True, try to cone search the source_id if not found in the lookup table
    """
    global gaia_id_df

    if pd.isna(text) or text == "":  # return NaN if the text is NaN or empty
        return pd.NA

    match = re_gaiadr3.search(text)
    if match:
        return int(match.group(1))
    else:
        if strict and not cone_search:  # if strict and not cone_search, then return NaN
            return pd.NA

    # otherwise try to find Gaia DR2
    if not cone_search and (
        match := re_gaiadr2.search(text)
    ):  # still need to make sure the source_id is valid
        # attempt to lookup the source_id in the lookup table but if not then try to cone search the source_id
        in_lookup_row = gaia_id_df[gaia_id_df["dr2_sourceid"] == int(match.group(1))]
        if len(in_lookup_row) > 0:
            # already in the lookup table, return the corresponding dr3_sourceid
            return int(in_lookup_row["dr3_sourceid"].values[0])
        # else:
        #     print(f"Gaia DR2 {match.group(1)} not found in DR3 in lookup table")
        if (
            len(
                Gaia.launch_job(
                    f"SELECT source_id FROM gaiadr3.gaia_source WHERE source_id={match.group(1)}"
                ).results
            )
            > 0
        ):
            # the case where the dr2==dr3, so no need to clone search
            # add the source_id to the lookup table
            gaia_id_df = pd.concat(
                [
                    gaia_id_df,
                    pd.DataFrame(
                        {
                            "dr2_sourceid": [int(match.group(1))],
                            "dr3_sourceid": [int(match.group(1))],
                        }
                    ),
                ]
            )
            return int(match.group(1))
        else:
            # in case we are upgrading to DR4, this need to be upgraded to DR3
            gaia_dr2 = Gaia.launch_job(
                f"SELECT designation, source_id, ra, dec, parallax, pmra, pmdec, phot_g_mean_mag, bp_rp, radial_velocity FROM gaiadr2.gaia_source WHERE source_id={match.group(1)}"
            ).results.filled(0)[0]

            # remember to change the epoch when updating to DR4
            query = rf"""SELECT TOP 10 designation, source_id, ra, dec, parallax, pmra, pmdec, ruwe, phot_g_mean_mag, bp_rp, radial_velocity,
DISTANCE(POINT('ICRS' , gaiadr3.gaia_source.ra, gaiadr3.gaia_source.dec), 
POINT('ICRS', 
COORD1(EPOCH_PROP_POS({gaia_dr2["ra"]}, {gaia_dr2["dec"]}, {gaia_dr2["parallax"]}, {gaia_dr2["pmra"]}, {gaia_dr2["pmdec"]}, {gaia_dr2["radial_velocity"]}, 2015.5, 2016.0)), 
COORD2(EPOCH_PROP_POS({gaia_dr2["ra"]}, {gaia_dr2["dec"]}, {gaia_dr2["parallax"]}, {gaia_dr2["pmra"]}, {gaia_dr2["pmdec"]}, {gaia_dr2["radial_velocity"]}, 2015.5, 2016.0)))) AS "target_separation (deg)"
FROM gaiadr3.gaia_source 
WHERE 
CONTAINS(
	POINT('ICRS',gaiadr3.gaia_source.ra,gaiadr3.gaia_source.dec),
	CIRCLE(
		'ICRS',
		COORD1(EPOCH_PROP_POS({gaia_dr2["ra"]}, {gaia_dr2["dec"]}, {gaia_dr2["parallax"]}, {gaia_dr2["pmra"]}, {gaia_dr2["pmdec"]}, {gaia_dr2["radial_velocity"]}, 2015.5, 2016.0)),
		COORD2(EPOCH_PROP_POS({gaia_dr2["ra"]}, {gaia_dr2["dec"]}, {gaia_dr2["parallax"]}, {gaia_dr2["pmra"]}, {gaia_dr2["pmdec"]}, {gaia_dr2["radial_velocity"]}, 2015.5, 2016.0)),
		{(15 * u.arcsecond).to(u.deg).value})
)=1 ORDER BY "target_separation (deg)"
"""
            tmp = Gaia.launch_job(query).results
            # tmp = Gaia.cone_search(f"Gaia DR2 {match.group(1)}", radius=15 * u.arcsecond).results
            if len(tmp) == 0:
                print(f"Gaia DR2 {match.group(1)} not found in DR3")
                return pd.NA
            else:
                # add the source_id to the lookup table
                gaia_id_df = pd.concat(
                    [
                        gaia_id_df,
                        pd.DataFrame(
                            {
                                "dr2_sourceid": [int(match.group(1))],
                                "dr3_sourceid": [int(tmp[0]["source_id"])],
                            }
                        ),
                    ]
                )
                return int(tmp[0]["source_id"])
    else:
        if cone_search and text:
            # try to cone search in case SIMBAD missed the Gaia DR number, stricter in radius to prevent false positive
            gaia_source = Gaia.cone_search(
                text.split("|")[1], radius=7.5 * u.arcsecond
            ).results
            if len(gaia_source) > 0:
                return int(gaia_source[0]["source_id"])
        return pd.NA


def parse_result(simbad_table):
    """
    This function will parse the result from the query:
    - Remove rows that are not stars
    - Add additional rows to the table for stars that are potentially in a binary system

    To attempt to resolve potential binary system, the logic is as follows:
    1. If the stars have V and J magnitude dimmer than 3.0, then it should have Gaia source_id
    2. When no Gaia source_id is found, query the children of the star because it might be a binary system resolved by Gaia hence no Gaia source_id
    3. If childen have no Gaia source_id, then assume this is not a binary system. Just for whatever reason Gaia did not observe the parent star
    4. If the star has children with source_id, then add the children to the table and remove the parent star

    Parameters
    ----------
    simbad_table : astropy.table.Table
        Table from the query result

    Returns
    -------
    astropy.table.Table
        Table with additional rows from the query result
    """
    simbad_table = simbad_table.to_pandas()
    simbad_table["otype"] = simbad_table["otype"].fillna("")
    simbad_table = simbad_table[simbad_table["otype"].apply(lambda x: "err" not in x)]

    # more strict on resolving the source_id in case of binary stars
    source_id = simbad_table["ids"].apply(lambda x: extract_gaia_number(x, strict=True))
    # these stars probably should have source_id, so we want to resolve possible binary stars
    hip_wo_source_id_idx = (
        pd.isna(source_id)
        & (simbad_table["V"].fillna(99.99) > 3.0)
        & (simbad_table["J"].fillna(99.99) > 2.0)
        & ~pd.isna(simbad_table["ids"])  # need to at least have some id
    )
    simbad_table_temp = simbad_table[hip_wo_source_id_idx]
    result = []
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for i, irow in tqdm.tqdm(
            simbad_table_temp.iterrows(),
            total=len(simbad_table_temp),
            desc="Resolving potential binary",
        ):
            # if children have no SAO/HD/HR id, then use the parent id
            parent_sao = re_sao.search(irow["ids"])
            parent_hd = re_hd.search(irow["ids"])
            parent_hr = re_hr.search(irow["ids"])

            tmp_query = custom_simbad.query_hierarchy(
                irow["main_id"],
                hierarchy="children",
                get_query_payload=True,
                detailed_hierarchy=False,
            )["QUERY"]
            tmp_hash = hashlib.sha1(tmp_query.encode("utf-8")).hexdigest()
            tmp_file = simbad_heriarchy_cache / f"{tmp_hash}.dat"
            # if not exist then query and save to file for caching
            if tmp_file.exists():
                temp = pd.read_csv(tmp_file)
            else:
                temp = custom_simbad.query_hierarchy(
                    irow["main_id"], hierarchy="children", detailed_hierarchy=False
                )
                temp = temp.to_pandas()
                temp["user_specified_id"] = irow["user_specified_id"].strip()
                source_id = temp["ids"].apply(
                    lambda x: extract_gaia_number(x, cone_search=True)
                )
                # add letter to the end of the user_specified_id, so first row in temp will end with "A", second row in temp will end with "B", etc
                for idx, row in temp.iterrows():
                    temp.loc[idx, "user_specified_id"] = (
                        f"{row['user_specified_id']}{chr(65 + idx)}"
                    )
                    # add user_specified_id to the table ids column
                    temp.loc[idx, "ids"] = (
                        f"{temp.loc[idx, 'user_specified_id']}|{temp.loc[idx, 'ids']}"
                    )
                    # add temp_source_id for each row to the ids column if it is not NaN
                    if not pd.isna(source_id[idx]):
                        temp.loc[idx, "ids"] = (
                            f"{temp.loc[idx, 'ids']}|Gaia DR3 {source_id[idx]}"
                        )
                temp.to_csv(tmp_file, index=False)
            source_id = temp["ids"].apply(lambda x: extract_gaia_number(x))
            if len(temp) <= 1:
                # in case some stars only have one component which is itself or no result
                continue
            # the second row and beyond should have source_id, if not then assume it is not a binary system
            # also if all of them has the same source_id, then assume it is not a binary system
            if pd.isna(source_id[1:]).any() or source_id.duplicated(keep=False).any():
                continue
            # add the parent SAO/HD/HR id to the ids column if it is not NaN
            temp["ids"] = temp["ids"].apply(
                lambda x: f"{x}|SAO {parent_sao.group(1)}" if parent_sao else x,
            )
            temp["ids"] = temp["ids"].apply(
                lambda x: f"{x}|HD {parent_hd.group(1)}" if parent_hd else x,
            )
            temp["ids"] = temp["ids"].apply(
                lambda x: f"{x}|HR {parent_hr.group(1)}" if parent_hr else x,
            )
            result.append(temp)
            simbad_table.drop(i, inplace=True)
    if len(result) > 0:
        # append list of result to simbad_table
        simbad_table = pd.concat([simbad_table, *result])

    # drop empty rows
    simbad_table = simbad_table.dropna(how="all")
    # reset the index
    simbad_table.reset_index(drop=True, inplace=True)
    # try to resolve the source_id for the remaining stars
    source_id = simbad_table["ids"].apply(lambda x: extract_gaia_number(x, strict=True))
    wo_source_id_idx = (
        pd.isna(source_id)
        & (simbad_table["V"].fillna(99.99) > 3.0)
        & ~pd.isna(simbad_table["ids"])  # need to at least have some id
    )
    # apply extract_gaia_number with cone_search=True to the remaining stars
    temp = simbad_table[wo_source_id_idx]
    for idx, row in tqdm.tqdm(
        temp.iterrows(), total=len(temp), desc="Resolving Gaia DR3 ID xmatch"
    ):
        time.sleep(1.0)  # sleep for 1.0 seconds to prevent rate limit
        try:
            simbad_table.loc[idx, "ids"] = (
                f"{row['ids']}|Gaia DR3 {extract_gaia_number(row['ids'], cone_search=True)}"
            )
        except NameResolveError as e:
            print(f"Failed to resolve {row['main_id']} due to {e}")
            continue
    # simbad_table.loc[wo_source_id_idx, "ids"] = simbad_table.loc[wo_source_id_idx, "ids"].apply(
    #     lambda x: f"{x}|Gaia DR3 {extract_gaia_number(x, cone_search=True)}"
    # )
    return simbad_table


def parse_cross_id(result):
    """
    Parse the cross id from the query result

    Parameters
    ----------
    result : pandas.DataFrame
        The result from the query
    """

    def extract_id(text, sid, group_id=1):
        if not pd.isna(text):  # check if not NaN
            if isinstance(sid, str):
                match = re.search(rf"{sid}\s+(\d+)([A-Z]?)", text)
            elif isinstance(sid, re.Pattern):
                match = sid.search(text)
            else:
                raise ValueError("sid must be either str or re.Pattern")
            if match and match.group(group_id) != "":
                return match.group(group_id).strip()
        return pd.NA

    def parse_component(text):
        if pd.isna(text) or text == "" or text == 0:
            return pd.NA
        else:
            return text

    df = pd.DataFrame(
        columns=["hip", "gaia_dr3", "component", "sao", "hd", "hr"], dtype="Int64"
    )
    df["component"] = df["component"].astype(pd.StringDtype())
    hip, sao, hd, hr = [
        result["ids"].apply(lambda x: extract_id(x, sid)).astype("Int64")
        for sid in [re_hip, re_sao, re_hd, re_hr]
    ]
    gaia = result["ids"].apply(lambda x: extract_gaia_number(x)).astype("Int64")
    component = (
        result["ids"]
        .apply(lambda x: parse_component(extract_id(x, "HIP", group_id=2)))
        .astype(pd.StringDtype())
    )

    # fill the dataframe
    df["hip"] = hip
    df["sao"] = sao
    df["hd"] = hd
    df["hr"] = hr
    df["gaia_dr3"] = gaia
    df["component"] = component

    return df


hip_with_binary = parse_result(hip_combined_table)
# deal with Sirius A and Sirius B, because SIMBAD said Sirius does not have children
hip_with_binary.loc[hip_with_binary["main_id"] == "* alf CMa", "ids"] = (
    "HIP 32349A|"
    + hip_with_binary.loc[hip_with_binary["main_id"] == "* alf CMa", "ids"]
)

result = custom_simbad.query_object("Sirius B")
result["ids"] = "HIP 32349B|" + result["ids"]
# cast to pandas and back to prevent potential dtype incompatibility issue
simbad_table = pd.concat([hip_with_binary, result.to_pandas()])
simbad_table = simbad_table.dropna(subset=["ids"])

# copy simbad_table to a new variable to prevent modifying the original table
simbad_table_db = simbad_table.copy()
cross_id_df = pd.concat([cross_id_df, parse_cross_id(simbad_table)])
simbad_table_db["hip"] = cross_id_df["hip"].fillna(0)
simbad_table_db["source_id"] = cross_id_df["gaia_dr3"].fillna(0)
simbad_table_db["componentid"] = cross_id_df["component"]
# replace the NaN with 0, A to 1, B to 2, etc
simbad_table_db["componentid"] = simbad_table_db["componentid"].apply(
    lambda x: 0 if pd.isna(x) else ord(x) - 64
)
simbad_table_db.to_csv(base_path / "hip_processed_with_binary.dat", index=False)
# drop rows with NaN "ids" column
cross_id_df = cross_id_df.dropna(how="all")

# find which SAO id is missing, should be SAO 1 to SAO max_sao_id. Only query the missing SAO id
existing_sao_ids = set(cross_id_df["sao"].dropna().astype(int))
all_sao_ids = set(range(1, len(sao_combined_table) + 1))
missing_sao_idx = list(all_sao_ids - existing_sao_ids)
# get the missing SAO id with corresponding row
missing_sao = sao_combined_table[[i - 1 for i in missing_sao_idx]]
sao_cross_id_df = parse_cross_id(missing_sao.to_pandas())
cross_id_df = pd.concat([cross_id_df, sao_cross_id_df])


# find which HD id is missing, should be HD 1 to HD max_hd_id. Only query the missing HD id
existing_hd_ids = set(cross_id_df["hd"].dropna().astype(int))
all_hd_ids = set(range(1, len(hd_combined_table) + 1))
missing_hd_idx = list(all_hd_ids - existing_hd_ids)
# get the missing HD id with corresponding row
missing_hd = hd_combined_table[[i - 1 for i in missing_hd_idx]]
cross_id_df = pd.concat([cross_id_df, parse_cross_id(missing_hd.to_pandas())])
# save the lookup table
gaia_id_df.to_csv(gaia_id_path, index=False)

Resolving Gaia DR3 ID xmatch: 100%|██████████| 566/566 [16:29<00:00,  1.75s/it]


Gaia DR2 239863001382455424 not found in DR3


In [ ]:
# drop rows with NaN "ids" column
cross_id_df = cross_id_df.dropna(how="all")
cross_id_df.to_csv(base_path / "cross_id_with_gaiaid.dat", index=False, sep="\t")
# if hip is missing, then set hip to the corresponding gaia id and then delete the gaia column
cross_id_df.loc[pd.isna(cross_id_df["hip"]), "hip"] = cross_id_df.loc[
    pd.isna(cross_id_df["hip"]), "gaia_dr3"
].values
cross_id_df = cross_id_df.drop(columns=["gaia_dr3"])
# if all sao, hd, hr is missing, then delete those rows. No point of keeping them
cross_id_df = cross_id_df[
    (
        ~pd.isna(cross_id_df["sao"])
        | ~pd.isna(cross_id_df["hd"])
        | ~pd.isna(cross_id_df["hr"])
    )
    & ~pd.isna(cross_id_df["hip"])
]

cross_id_df = cross_id_df.sort_values(["hip", "component"]).astype(
    {"component": "object"}
)

# cross_id_df.to_csv(
#     base_path / "cross-id.dat",
#     index=False,
#     sep="\t",
#     header=["#" + col for col in cross_id_df.columns],
# )

# change component A, B, C... to 1, 2, 3...
cross_id_df["component"] = cross_id_df["component"].apply(
    lambda x: ord(x) - ord("A") + 1 if x == x else 0
)
cross_id_df.fillna(0, inplace=True)
cross_id_df = cross_id_df.astype("Int64")
record_size = 24
bdata = bytearray(record_size * len(cross_id_df))

for i, row in enumerate(cross_id_df.itertuples()):
    byte_data = struct.pack("Qiiii", *row[1:])
    bdata[i * record_size : (1 + i) * record_size] = byte_data

with open(base_path / "cross-id.cat", "wb") as f:
    f.write(bdata)